## Build Data

In [ ]:
import sys
sys.path.append('..')
filename = 'samples_zhihu_pro.jsonl'
lang = 'Chinese'

In [ ]:
import jsonlines
samples_pro = []
with jsonlines.open(filename) as reader:
    for item in reader:
        samples_pro.append(item)

In [ ]:
import re
from utils.lang_utils import cut_sent
i = 500
j = 1
for k in samples_pro[i]['memory_maze']:
    if k.startswith(f'sl{j}'):
        for sent in cut_sent(samples_pro[i]['memory_maze'][k]['content']):
            print(sent)
        print("*****")

In [ ]:
# print(samples_pro[0]['passage'])

In [ ]:
samples_pro = [sample for sample in samples_pro if 'memory_maze' in sample]
len(samples_pro)

In [ ]:
from agents.memory_tree import MemoryTree
from utils.file_utils import jsonl_filename_suffix
def build_lines_one(sample, end_idx=10):
    mem = MemoryTree(sample['memory_maze'])
    sents = mem.get_originals()
    sents = ['\n'.join(cut_sent(s)) for s in sents if s.strip()]
    # print(len(sents))
    summary, _ = mem.get_summary(end_idx=end_idx)
    summary = [s.replace('\n\n', '\n') for s in summary]
    # print(len(summary))
    ret = summary + sents[end_idx:end_idx+10-len(summary)]
    return ret
build_lines_one(samples_pro[11], 40)

In [ ]:
def build_sys(j):
    sys = "Style: \n{0}\n\nBackground: \n{1}\n\nPlot: \n{2}\n\nProtagonists: \n{3}".format(j['style'], j['background'], j['plot'], j['protagonists'])
    sys2 = 'We are writing this story block by block. Each block is around 200 words. Please follow the setting and style. The outline should be finished in around 10k words. '
    return sys+'\n\n'+sys2
print(build_sys(samples_pro[0]))

In [ ]:
from utils.agent_utils import lines2messages
def lines2turns(lines, sys, lang='Chinese'):
    turns1 = lines2messages(lines, sys, lang)
    turns2 = lines2messages(lines[:-1], sys, lang)
    return turns1, turns2
turns1, turns2 = lines2turns(build_lines_one(samples_pro[11], 40), build_sys(samples_pro[11]), lang)
turns1

In [ ]:
print(turns1[0]['content'])

In [ ]:
messages_lst = []
import random
from tqdm import tqdm
i = 0
for sample in tqdm(samples_pro):
    i += 1
    mem = MemoryTree(sample['memory_maze'], lang=lang)
    end_idxs = random.sample(range(2, len(mem.get_originals())-1), min(len(mem.get_originals())-3, 10))
    for end_idx in end_idxs:
        turns1, turns2 = lines2turns(build_lines_one(sample, end_idx), build_sys(sample), lang)
        messages_lst.append(turns1)
        messages_lst.append(turns2)

In [ ]:
print(len(messages_lst))
messages_lst[100]

In [ ]:
import json
def lst2jsonl(messages_lst, fn):
    with open(fn, "w") as f:
        for j in messages_lst:
            f.write(json.dumps({"messages": j}) + "\n")
lst2jsonl(messages_lst, jsonl_filename_suffix(filename, 'mem'))

In [ ]:
def deduplicate_jsonl(input_file, output_file):
    # Create a set to store unique lines of the JSONL file
    unique_lines = set()

    # Open the input file in read mode
    with open(input_file, 'r') as f:
        # Read each line from the input file
        for line in f:
            # Convert the line to a dictionary
            data = json.loads(line.strip())
            # Convert the dictionary back to a string
            json_string = json.dumps(data, sort_keys=True)
            # Add the string to the set
            unique_lines.add(json_string)

    # Open the output file in write mode
    with open(output_file, 'w') as f:
        # Write each unique line to the output file
        for line in unique_lines:
            f.write(line + '\n')

# Example usage
deduplicate_jsonl(jsonl_filename_suffix(filename, 'mem'), jsonl_filename_suffix(jsonl_filename_suffix(filename, 'mem'), 'deduplicated'))

In [ ]:
import random
def random_sample_half(input_file, output_file):
    # Read lines from input file
    with open(input_file, 'r') as infile:
        lines = infile.readlines()

    # Get half the length of lines
    half_length = len(lines) // 2

    # Randomly select half the lines
    sampled_lines = random.sample(lines, half_length)

    print(len(lines), len(sampled_lines))

    # Save sampled lines to output file
    with open(output_file, 'w') as outfile:
        outfile.writelines(sampled_lines)
random_sample_half(jsonl_filename_suffix(jsonl_filename_suffix(filename, 'mem'), 'deduplicated'), jsonl_filename_suffix(jsonl_filename_suffix(jsonl_filename_suffix(filename, 'mem'), 'deduplicated'), 'half'))